In [1]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración visual
sns.set(style="whitegrid")  # estilo de Seaborn
plt.rcParams['figure.figsize'] = (12, 6)  # tamaño por defecto de gráficos

# Configuración para mostrar todos los resultados de un DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)